Finding the toal number of enteries in the dataset 

In [1]:
import gzip
count=0
for i in range(0,500):
    num=str(i)
    zero_fill_num=num.zfill(5)
    with gzip.open('/ichec/work/mucom001c/GoogleSvrLog/clusterdata-2011-1/task_usage/part-'+zero_fill_num+'-of-00500.csv.gz', 'rb') as f:
        for line in f:
            line=line.decode()
            line=line.split(',')
            t=line[0]
            count+=1
print(count)

1232799308


Total number of enteries in the dataset are :1232799308

Average entry per file :2465598.6

Data attributes in the file :20

## finding the size of the file 

In [2]:
from pathlib import Path
import gzip
size=0
for i in range(0,500):
    num=str(i)
    zero_fill_num=num.zfill(5)
    size+=Path('/ichec/work/mucom001c/GoogleSvrLog/clusterdata-2011-1/task_usage/part-'+zero_fill_num+'-of-00500.csv.gz').stat().st_size
print(size)

39385409295


The total size of zipped file is about : 39385409295 byte or 39.386 GB

In [3]:
import os
def setup_my_environment():
    import os
    
def setenv(var, val):
    os.environ[var] = val

def prepend_path(var, val):
    old_val = os.environ.get(var, '')
    os.environ[var] = val + ":" + old_val
    
def setup_java():
    PKG_ROOT='/ichec/packages/java/8'
    setenv('JAVA_PATH', PKG_ROOT)
    setenv('JAVA_HOME', PKG_ROOT)
    prepend_path('PATH', PKG_ROOT + '/bin')
    prepend_path('MANPATH', PKG_ROOT + '/man')
    prepend_path('CPATH', PKG_ROOT + '/include')
    
def setup_spark():
    PKG_ROOT='/ichec/packages/spark/2.3.3/kay/spark-2.3.3'
    setenv('SPARK_DIST_CLASSPATH', PKG_ROOT + 'spark-2.3.3-bin-kay-spark')
    prepend_path('PATH', PKG_ROOT + PKG_ROOT + 'spark-2.3.3-bin-kay-spark/bin')
setup_java()
setup_spark()
setup_my_environment()
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

 

spark = SparkSession \
    .builder \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "5g") \
    .config("spark.executor.instances","20")\
    .config("spark.executor.cores","2")\
    .config("spark.worker.instances","20")\
    .config("spark.memory.offHeap.enabled","true")\
    .config("spark.memory.offHeap.size","25g")\
    .config("spark.debug.maxToStringsFields","100")\
    .appName("GoogleSvrLog") \
    .getOrCreate()

In [4]:
df = spark.read.csv("/ichec/work/mucom001c/GoogleSvrLog/clusterdata-2011-1/task_usage/*.gz")
df6=df.select('_c0','_c1','_c2','_c3','_c4')
df6=df6.selectExpr('_c0 as start_time','_c1 as end_time','_c2 as job_id','_c3 as task_index','_c4 as machine_id')

In [5]:
df6.show()

+------------+------------+----------+----------+----------+
|  start_time|    end_time|    job_id|task_index|machine_id|
+------------+------------+----------+----------+----------+
|807404000000|807600000000|5822896270|        84|  82731175|
|807404000000|807406000000|5912313637|        99|   1269850|
|807404000000|807405000000|5912313637|       374| 367886065|
|807404000000|807462000000|6320750684|       306|    660385|
|807404000000|807600000000|6320750684|       313| 294901680|
|807404000000|807600000000|6321705030|         1|4825194215|
|807404000000|807600000000|6321705030|       356|  38728345|
|807404000000|807600000000|6321705030|       694| 376217870|
|807404000000|807600000000|6321705030|       958|4820139430|
|807404000000|807600000000|6321910142|       162| 367886065|
|807404000000|807600000000|6321910142|       797|    705825|
|807404000000|807533000000|6321910142|      1001| 336058605|
|807404000000|807600000000|6321910142|      1008|   6608775|
|807404000000|8076000000

In [17]:
from pyspark.sql.functions import *

df5new = df6.withColumn('start_time', from_unixtime(df6.start_time, "HH"))


In [18]:
df5new.show(5)

+----------+------------+----------+----------+----------+
|start_time|    end_time|    job_id|task_index|machine_id|
+----------+------------+----------+----------+----------+
|        17|807600000000|5822896270|        84|  82731175|
|        17|807406000000|5912313637|        99|   1269850|
|        17|807405000000|5912313637|       374| 367886065|
|        17|807462000000|6320750684|       306|    660385|
|        17|807600000000|6320750684|       313| 294901680|
+----------+------------+----------+----------+----------+
only showing top 5 rows



In [19]:
df5new = df5new.withColumn('end_time', from_unixtime(df5new.end_time, "HH"))

In [20]:
df5new.show(5)

+----------+--------+----------+----------+----------+
|start_time|end_time|    job_id|task_index|machine_id|
+----------+--------+----------+----------+----------+
|        17|      05|5822896270|        84|  82731175|
|        17|      21|5912313637|        99|   1269850|
|        17|      07|5912313637|       374| 367886065|
|        17|      01|6320750684|       306|    660385|
|        17|      05|6320750684|       313| 294901680|
+----------+--------+----------+----------+----------+
only showing top 5 rows



In [21]:
df5new.select('task_index').distinct().show(35)

+----------+
|task_index|
+----------+
|      8304|
|     10351|
|     15555|
|      3414|
|      6613|
|      7273|
|      9586|
|     13442|
|     15574|
|      1090|
|      1436|
|      1512|
|      4937|
|     10436|
|     13610|
|      8433|
|      9009|
|     14887|
|      2294|
|      6240|
|      9993|
|     11078|
|     15271|
|      1159|
|      2904|
|       829|
|     13772|
|     14838|
|     14899|
|     13865|
|      1572|
|      3606|
|     11332|
|      3959|
|      6731|
+----------+
only showing top 35 rows

